In [1]:
import pandas as pd
import re
import sentencepiece as spm
import copy
import torch

In [9]:
train_set = pd.read_excel('/share_v1/Tianqiao/NLG/question_generation/equation2text/case_study/自己编的测试集.xlsx')

In [10]:
def process_scene(scene):
    scene = scene.replace(' ','')
    map_scene = {}
    edge_0, edge_1 = re.split(r'\),|\)，',scene)
    ent0, ent1 = edge_0.split('(')
    map_scene[ent0] = ent1
    
    ent2, ent3 = edge_1[:-1].split('(')
    map_scene[ent2] = ent3
    return map_scene
def process_var(var):
    map_var = {}
    edge_0, edge_1 = re.split(r'\),|\)，',var)
    ent0, ent1 = edge_0.split('(')
    map_var[ent1] = ent0.replace(' ','')
    ent2, ent3 = edge_1[:-1].split('(')
    map_var[ent3] = ent2.replace(' ','')
    return map_var
def process_mr(mr, var):
    splited_mr = mr.split(',')
    mr_dict = {}
    for one in splited_mr:
        ent0, ent1 = one.split('[')
        mr_dict[ent0] = ent1[:-1]
    processed_var = process_var(var)
    mr_dict['x_ent'] = processed_var['x']
    mr_dict['y_ent'] = processed_var['y']
    return mr_dict
def fenge_B006(text):
    a = re.split(r'(\+|,|-|\*|/|=|!=|>=|<=|>|<|\(|\)|\[|\]|\\|\|\b\d+\w*\b|[\u4E00-\u9FA5]|\②|\①|\③|\④|\㉖|\:)', text)
    return [x.strip() for x in a if x!='']

In [11]:
# 移项，如果变量在等号右边，把它移到左边
def transfer_var(equation):
    splited_equ = fenge_B006(equation)
    equ_index = splited_equ.index('=')
    equ_right = splited_equ[equ_index+1:]
    equ_left = splited_equ[:equ_index]
    left_yubei = []
    right_yubei = []
    for idx, ele in enumerate(equ_left):
        if ('x' not in ele) and ('y' not in ele) and (ele not in ['+','-','*','/']):
            # 有数字
            if idx==0:
                right_yubei += ['-', ele]
                equ_left = equ_left[1:]
            else:
                if equ_left[idx-1] == '-':
                    right_yubei += ['+', ele]
                    equ_left = equ_left[:idx-1] + equ_left[idx+1:]
                elif equ_left[idx-1] == '+':
                    right_yubei += ['-', ele]
                    equ_left = equ_left[:idx-1] + equ_left[idx+1:]
    
    equ_right = equ_right + right_yubei
    if (equation.find('x') > equation.find('=')) or (equation.find('y') > equation.find('=')):
        # 先把全部right元素移到左边
        for idx, ele in enumerate(equ_right):
            if ('x' in ele) or ('y' in ele):
                if idx ==0:
                    left_yubei += ['-',ele]
                    if len(equ_right) == 1:
                        equ_right[0] = '0'
                    else:
                        equ_right = equ_right[2:]
                else:
                    if equ_right[idx-1] == '-':
                        left_yubei += ['+',ele]
                        equ_right = equ_right[:idx-1] + equ_right[idx+1:]
                    elif equ_right[idx-1] == '+':
                        left_yubei += ['-', ele]
                        equ_right = equ_right[:idx-1] + equ_right[idx+1:]
    else:
        return fenge_B006(equation)
    
#     print(equ_left)
#     print(left_yubei)
#     print(equ_right)
#     print(right_yubei)
    equ_left = equ_left + left_yubei
    #equ_right = equ_right + right_yubei
    return equ_left + ['='] + equ_right

In [12]:
guanxi1_trans, guanxi2_trans = [], []
for idx, row in train_set.iterrows():
    guanxi1 = row['关系1']
    guanxi2 = row['关系2']
    guanxi1_trans.append(''.join(transfer_var(guanxi1)))
    guanxi2_trans.append(''.join(transfer_var(guanxi2)))
train_set['关系1_trans'] = guanxi1_trans
train_set['关系2_trans'] = guanxi2_trans

In [13]:
def x_fir_y_sec(equation):
    equ_left = equation[:equation.find('=')]
    equ_right = equation[equation.find('='):]
    splited_left = fenge_B006(equation[:equation.find('=')])
    if equ_left.find('x') > equ_left.find('y'):
        # x的表示在y的后面，不合理
        if len(splited_left) == 3:
            splited_left = splited_left[-2:] + ['+']+[splited_left[0]]
            if splited_left[0] == '+':
                splited_left = splited_left[1:]
        elif len(splited_left) == 4:
            splited_left = splited_left[-2:] + splited_left[:2]
            if splited_left[0] == '+':
                splited_left = splited_left[1:]
    return ''.join(splited_left) + equ_right

In [14]:
train_set['关系1_trans'] = train_set['关系1_trans'].map(lambda x: x_fir_y_sec(x))
train_set['关系2_trans'] = train_set['关系2_trans'].map(lambda x: x_fir_y_sec(x))

In [15]:
def get_equation_mr(equation1, equation2):
    equ1_left = equation1[:equation1.find('=')]
    equ1_right = equation1[equation1.find('=')+1:]
    splited_left_1 = fenge_B006(equ1_left)
    splited_right_1 = fenge_B006(equ1_right)
    one_mr = []
    # 先处理第一个式子的左半部分
    if len(splited_left_1) == 3:
        # 处理x部分
        if splited_left_1[0] =='x':
            pass
        else:
            one_mr.append('eq1_x_index[{}]'.format(splited_left_1[0].replace('x','')))
        # 处理中间的那个运算符
        one_mr.append('eq1_left_sym2[{}]'.format(splited_left_1[1]))
        # 处理y部分
        if splited_left_1[2] =='y':
            pass
        else:
            one_mr.append('eq1_y_index[{}]'.format(splited_left_1[2].replace('y','')))
    elif len(splited_left_1) == 4:
        # 有四个部分，看起来都要做填充
        # 第一个部分的符号
        one_mr.append('eq1_left_sym1[{}]'.format(splited_left_1[0]))
        if splited_left_1[1] =='x':
            pass
        else:
            one_mr.append('eq1_x_index[{}]'.format(splited_left_1[1].replace('x','')))
        # 第二个部分的符号
        one_mr.append('eq1_left_sym2[{}]'.format(splited_left_1[2]))
        # 处理y部分
        if splited_left_1[3] =='y':
            pass
        else:
            one_mr.append('eq1_y_index[{}]'.format(splited_left_1[3].replace('y','')))
    else:
        raise 'wrong input equation1, check preprocessing'
    
    # 处理第一个式子的右半部分
    if len(splited_right_1) == 1:
        # 只有一个实数，应该大部分都是如此
        one_mr.append('eq1_right_num1[{}]'.format(splited_right_1[0]))
    elif len(splited_right_1) == 3:
        # 有实数的计算存在，好吧，那就算一算吧
        one_mr.append('eq1_right_num1[{}]'.format(splited_right_1[0]))
        one_mr.append('eq1_right_sym[{}]'.format(splited_right_1[1]))
        one_mr.append('eq1_right_num2[{}]'.format(splited_right_1[2]))
    
    
    equ2_left = equation2[:equation2.find('=')]
    equ2_right = equation2[equation2.find('=')+1:]
    splited_left_2 = fenge_B006(equ2_left)
    splited_right_2 = fenge_B006(equ2_right)
    
    # 先处理第二个式子的左半部分
    if len(splited_left_2) == 3:
        # 处理x部分
        if splited_left_2[0] =='x':
            pass
        else:
            one_mr.append('eq2_x_index[{}]'.format(splited_left_2[0].replace('x','')))
        # 处理中间的那个运算符
        one_mr.append('eq2_left_sym2[{}]'.format(splited_left_2[1]))
        # 处理y部分
        if splited_left_2[2] =='y':
            pass
        else:
            one_mr.append('eq2_y_index[{}]'.format(splited_left_2[2].replace('y','')))
    elif len(splited_left_2) == 4:
        # 有四个部分，看起来都要做填充
        # 第一个部分的符号
        one_mr.append('eq2_left_sym1[{}]'.format(splited_left_2[0]))
        if splited_left_2[1] =='x':
            pass
        else:
            one_mr.append('eq2_x_index[{}]'.format(splited_left_2[1].replace('x','')))
        # 第二个部分的符号
        one_mr.append('eq2_left_sym2[{}]'.format(splited_left_2[2]))
        # 处理y部分
        if splited_left_2[3] =='y':
            pass
        else:
            one_mr.append('eq2_y_index[{}]'.format(splited_left_2[3].replace('y','')))
    else:
        raise 'wrong input equation2, check preprocessing'
    
    # 处理第一个式子的右半部分
    if len(splited_right_2) == 1:
        # 只有一个实数，应该大部分都是如此
        one_mr.append('eq2_right_num1[{}]'.format(splited_right_2[0]))
    elif len(splited_right_2) == 3:
        # 有实数的计算存在，好吧，那就算一算吧
        one_mr.append('eq2_right_num1[{}]'.format(splited_right_2[0]))
        one_mr.append('eq2_right_sym[{}]'.format(splited_right_2[1]))
        one_mr.append('eq2_right_num2[{}]'.format(splited_right_2[2]))
    return ','.join(one_mr)

In [16]:
new_mr = []
for idx, row in train_set.iterrows():
    equ1, equ2 = row['关系1_trans'], row['关系2_trans']
    new_mr.append(get_equation_mr(equ1, equ2))
train_set['mr'] = new_mr

In [17]:
def expand_mr(var, scene, now_mr, tou_info, jiao_info):
    processed_var, processed_scene = process_var(var), process_scene(scene)
    mr_dict = process_mr(now_mr, var)
    mr_dict['scene_x'] = processed_scene[mr_dict['x_ent']]
    mr_dict['scene_y'] = processed_scene[mr_dict['y_ent']]
    mr_dict['head_info_entity'] = eval(tou_info)['entity']
    mr_dict['head_info_unit'] = eval(tou_info)['unit']
    mr_dict['jiao_info_unit'] = eval(jiao_info)['unit']
    mr_dict['jiao_info_entity'] = eval(jiao_info)['entity']
    return mr_dict

In [18]:
class Constants:  
    def __init__(self):
        self.BOS_WORD = '<s>'
        self.EOS_WORD = '</s>'
        self.PAD_WORD = '<pad>'
        self.NONE_WORD = '<blank>'
        self.UNK_WORD = '<unk>'
        self.eq1_x_index_WORD = 'eq_one_x_index'
        self.eq2_x_index_WORD = 'eq_two_x_index'
        self.eq1_y_index_WORD = 'eq_one_y_index'
        self.eq2_y_index_WORD = 'eq_two_y_index'
        self.eq1_right_num1_WORD = 'eq_one_right_num_one'
        self.eq2_right_num1_WORD = 'eq_two_right_num_one'
        self.eq1_right_num2_WORD = 'eq_one_right_num_two'
        self.eq2_right_num2_WORD = 'eq_two_right_num_two'
        self.x_entity_WORD = 'x_entity'
        self.y_entity_WORD = 'y_entity'
        self.head_info_unit_WORD = 'head_info_unit'
        self.jiao_info_unit_WORD = 'jiao_info_unit'
        self.jiao_info_entity_WORD = 'jiao_info_entity'
        self.head_info_entity_WORD = 'head_info_entity'
        
        self.PAD = 0
        self.UNK = 1
        self.BOS = 2
        self.EOS = 3
        self.eq1_x_index = 4
        self.eq2_x_index = 5
        self.eq1_y_index = 6
        self.eq2_y_index = 7
        self.eq1_right_num1 = 8
        self.eq2_right_num1 = 9
        self.eq1_right_num2 = 10
        self.eq2_right_num2 = 11
        self.x_entity = 12
        self.y_entity = 13
        self.head_info_unit = 14
        self.jiao_info_unit = 15
        self.jiao_info_entity =16
        self.none=18
        self.head_info_entity =17
Constants = Constants()

In [19]:
MR_FIELDS =['方程二右边数字一', '方程一右边数字一', 'x_entity','y_entity', 'head信息_entity',
    '脚信息_entity', 'head信息_unit', '脚信息_unit', 
    '方程一y系数', '方程二y系数', '方程一x系数', '方程二x系数','方程一右边数字二','方程二右边数字二',
    '方程一左运算符1','方程一左运算符2','方程一右运算符','方程二左运算符1','方程二左运算符2','方程二右运算符',
           '场景_x','场景_y']
MR_FIELDS = ['eq2_right_num1', 'eq1_right_num1','x_ent','y_ent','head_info_entity','jiao_info_entity',
             'head_info_unit','jiao_info_unit', 'eq1_y_index', 'eq2_y_index', 'eq1_x_index', 'eq2_x_index', 'eq1_right_num2',
              'eq2_right_num2', 'eq1_left_sym1', 'eq1_left_sym2', 'eq1_right_sym', 'eq2_left_sym1','eq2_left_sym2',
            'eq2_right_sym','scene_x','scene_y']

MR_KEYMAP = dict((key, idx) for idx, key in enumerate(MR_FIELDS))
MR_KEY_NUM = len(MR_FIELDS)

lex_fields = ['方程二右边数字一', '方程一右边数字一', 'x_entity', 'y_entity', 'head信息_entity', '脚信息_entity',
              'head信息_unit','脚信息_unit',
    '方程一y系数', '方程二y系数', '方程一x系数', '方程二x系数', '方程一右边数字二','方程二右边数字二']
lex_fields = ['eq2_right_num1', 'eq1_right_num1','x_ent','y_ent','head_info_entity','jiao_info_entity',
             'head_info_unit','jiao_info_unit', 'eq1_y_index', 'eq2_y_index', 'eq1_x_index', 'eq2_x_index', 'eq1_right_num2',
              'eq2_right_num2']

lex_tar = [Constants.eq2_right_num1_WORD, Constants.eq1_right_num1_WORD, Constants.x_entity_WORD, Constants.y_entity_WORD, Constants.head_info_entity_WORD, Constants.jiao_info_entity_WORD,
    Constants.head_info_unit_WORD, Constants.jiao_info_unit_WORD,
    Constants.eq1_y_index_WORD,Constants.eq2_y_index_WORD,
    Constants.eq1_x_index_WORD, Constants.eq2_x_index_WORD, Constants.eq1_right_num2_WORD, Constants.eq2_right_num2_WORD]

lex_keymap = dict((key, idx) for idx, key in enumerate(lex_fields))

In [42]:
tou = list(train_set["头信息"])
jiao = list(train_set["脚信息"])
scene = list(train_set["scene"])
bian_liang = list(train_set["变量"])

In [43]:
for i in range(len(tou)):
    tou[i] = tou[i].replace("\xa0","")
    jiao[i] = jiao[i].replace("\xa0","")
    scene[i] = scene[i].replace("\xa0","")
    bian_liang[i] = bian_liang[i].replace("\xa0","")

In [44]:
train_set["头信息"] = tou
train_set["脚信息"] = jiao
train_set["scene"] = scene
train_set["变量"] = bian_liang

In [49]:
seq2seq_train_mr = []
for idx, row in train_set.iterrows():
    seq2seq_train_mr.append(expand_mr(row['变量'], row['scene'], row['mr'], row['头信息'], row['脚信息']))
train_set['seq2seq_mr'] = seq2seq_train_mr

In [48]:
train_set.loc[51,"scene"] = "大床房(住宿),标准间(住宿)"

In [46]:
row

关系1                                                     x-y=12
关系2                                                   2x-4y=60
变量                                               大床房(x),标准间(y)
scene                                         热巧克力(食物),柠檬水(食物)
头信息                          {'entity':'','num':'','unit':'间'}
脚信息                          {'entity':'','num':'','unit':'人'}
new_type                                                 scene
关系1_trans                                               x-y=12
关系2_trans                                             2x-4y=60
mr           eq1_left_sym2[-],eq1_right_num1[12],eq2_x_inde...
Name: 51, dtype: object

In [50]:
class Constants:  
    def __init__(self):
        self.BOS_WORD = '<s>'
        self.EOS_WORD = '</s>'
        self.PAD_WORD = '<pad>'
        self.NONE_WORD = '<blank>'
        self.UNK_WORD = '<unk>'
        self.eq1_x_index_WORD = 'eq_one_x_index'
        self.eq2_x_index_WORD = 'eq_two_x_index'
        self.eq1_y_index_WORD = 'eq_one_y_index'
        self.eq2_y_index_WORD = 'eq_two_y_index'
        self.eq1_right_num1_WORD = 'eq_one_right_num_one'
        self.eq2_right_num1_WORD = 'eq_two_right_num_one'
        self.eq1_right_num2_WORD = 'eq_one_right_num_two'
        self.eq2_right_num2_WORD = 'eq_two_right_num_two'
        self.x_entity_WORD = 'x_entity'
        self.y_entity_WORD = 'y_entity'
        self.head_info_unit_WORD = 'head_info_unit'
        self.jiao_info_unit_WORD = 'jiao_info_unit'
        self.jiao_info_entity_WORD = 'jiao_info_entity'
        self.head_info_entity_WORD = 'head_info_entity'
        
        self.PAD = 0
        self.UNK = 1
        self.BOS = 2
        self.EOS = 3
        self.eq1_x_index = 4
        self.eq2_x_index = 5
        self.eq1_y_index = 6
        self.eq2_y_index = 7
        self.eq1_right_num1 = 8
        self.eq2_right_num1 = 9
        self.eq1_right_num2 = 10
        self.eq2_right_num2 = 11
        self.x_entity = 12
        self.y_entity = 13
        self.head_info_unit = 14
        self.jiao_info_unit = 15
        self.jiao_info_entity =16
        self.none=18
        self.head_info_entity =17
Constants = Constants()

In [51]:
all_keys = set()
for idx, row in train_set.iterrows():
    seq_mr = row['seq2seq_mr']
    for one_key in list(seq_mr.keys()):
        all_keys.add(one_key)

In [52]:
MR_FIELDS =['方程二右边数字一', '方程一右边数字一', 'x_entity','y_entity', 'head信息_entity',
    '脚信息_entity', 'head信息_unit', '脚信息_unit', 
    '方程一y系数', '方程二y系数', '方程一x系数', '方程二x系数','方程一右边数字二','方程二右边数字二',
    '方程一左运算符1','方程一左运算符2','方程一右运算符','方程二左运算符1','方程二左运算符2','方程二右运算符',
           '场景_x','场景_y']
MR_FIELDS = ['eq2_right_num1', 'eq1_right_num1','x_ent','y_ent','head_info_entity','jiao_info_entity',
             'head_info_unit','jiao_info_unit', 'eq1_y_index', 'eq2_y_index', 'eq1_x_index', 'eq2_x_index', 'eq1_right_num2',
              'eq2_right_num2', 'eq1_left_sym1', 'eq1_left_sym2', 'eq1_right_sym', 'eq2_left_sym1','eq2_left_sym2',
            'eq2_right_sym','scene_x','scene_y']

MR_KEYMAP = dict((key, idx) for idx, key in enumerate(MR_FIELDS))
MR_KEY_NUM = len(MR_FIELDS)

lex_fields = ['方程二右边数字一', '方程一右边数字一', 'x_entity', 'y_entity', 'head信息_entity', '脚信息_entity',
              'head信息_unit','脚信息_unit',
    '方程一y系数', '方程二y系数', '方程一x系数', '方程二x系数', '方程一右边数字二','方程二右边数字二']
lex_fields = ['eq2_right_num1', 'eq1_right_num1','x_ent','y_ent','head_info_entity','jiao_info_entity',
             'head_info_unit','jiao_info_unit', 'eq1_y_index', 'eq2_y_index', 'eq1_x_index', 'eq2_x_index', 'eq1_right_num2',
              'eq2_right_num2']

lex_tar = [Constants.eq2_right_num1_WORD, Constants.eq1_right_num1_WORD, Constants.x_entity_WORD, Constants.y_entity_WORD, Constants.head_info_entity_WORD, Constants.jiao_info_entity_WORD,
    Constants.head_info_unit_WORD, Constants.jiao_info_unit_WORD,
    Constants.eq1_y_index_WORD,Constants.eq2_y_index_WORD,
    Constants.eq1_x_index_WORD, Constants.eq2_x_index_WORD, Constants.eq1_right_num2_WORD, Constants.eq2_right_num2_WORD]

lex_keymap = dict((key, idx) for idx, key in enumerate(lex_fields))

In [53]:
import sentencepiece as spm
sp = spm.SentencePieceProcessor()
sp.load('./processed_data/encoded_with_rev.model')

True

In [54]:
def tokenize_word(text, sp, lex_list=None):
    words = []
    if lex_list:
        for word, tar in zip(lex_list, lex_tar):
            if word is not None:
                text = text.replace(word, tar)
    for frag in sp.EncodeAsPieces(text):
        words.append(frag)
    return words
def get_after_text(text, lex_list):
    if lex_list:
        for word, tar in zip(lex_list, lex_tar):
            if word is not None:
                text = text.replace(word, tar)
    return text

In [55]:
## 这个MR就是我们的seq了哈，要记住
def process_jitu_mr_type(mr_dict):
    mr_data = [Constants.NONE_WORD] * MR_KEY_NUM
    
    # hold lexicalized variants of key in lex_fields
    lex = [None, None, None, None, None, None,None, None, None, None, None, None, None, None]
    for idx, item in mr_dict.items():
        key, raw_val = idx, item
        if raw_val != '':
            key_idx = MR_KEYMAP[key]
            
            if key in lex_keymap:
                lex[lex_keymap[key]] = raw_val

            if key in lex_fields:
                if key == 'eq1_right_num1':
                    if mr_dict[key] == '0':
                        mr_data[key_idx] = '0'
                    else:
                        mr_data[key_idx] = lex_tar[lex_keymap[key]]
                elif key == 'eq2_right_num1':
                    if mr_dict[key] == '0':
                        mr_data[key_idx] = '0'
                    else:
                        mr_data[key_idx] = lex_tar[lex_keymap[key]]
                else:
                    mr_data[key_idx] = lex_tar[lex_keymap[key]]
            else:
                mr_data[key_idx] = raw_val
    return mr_data, lex

In [56]:
def read_insts_from_train_file(inst_file, sp, max_sen_len=None, mr_fields =MR_FIELDS,spec_field = lex_tar):
    """
    max_sen_len: how many word in one sentence
    spec_field: slor with lexicalized variants
    :return: mr data, lex: [as top], target
    """
    raw_data_src, raw_data_tgt = [],[]
    lexicalization = []
    trimmed_sent_count = 0
    for idx, row in inst_file.iterrows():
        mr, text = row['seq2seq_mr'], row['ref']
        mr_data_src, this_lex = process_jitu_mr_type(mr)
        this_text = tokenize_word(text, sp, this_lex)
        raw_data_src.append(mr_data_src)
        lexicalization.append(this_lex)
        if max_sen_len is not None:
            if len(this_text) > max_sen_len:
                this_text = this_text[:max_sen_len]
                trimmed_sent_count +=1
        
        if this_text:
            raw_data_tgt += [this_text + [Constants.EOS_WORD]]
        else:
            raw_data_tgt += [None]
    print('[Info] Get {} instances'.format(len(raw_data_tgt)))
    
    if trimmed_sent_count > 0:
        print('[Warning] {} instances are trimmed to the max sentence length {}.'.format(trimmed_sent_count,max_sen_len))
    return raw_data_src, lexicalization, raw_data_tgt

In [2]:
data = torch.load('/share_v1/Tianqiao/NLG/question_generation_NAACL/processed_data/new_seq2seq_data.pt')
word2idx = data['dict']['src']

In [8]:
data['train'].keys()

dict_keys(['src', 'tgt', 'lex'])

In [10]:
len(data['train']['src']) + len(data['dev']['src']) + len(data['test']['src'])

5447

In [ ]:
5447/6053

In [11]:
biaozhu = pd.read_excel('检查标注.xlsx')

(6041, 11)

In [59]:
tmp = []
for i in range(len(train_set)):
    tmp.append("")

In [60]:
train_set["ref"] = tmp

In [61]:
train_seq2seq_src, train_seq2seq_lex, train_seq2seq_tgt = read_insts_from_train_file(train_set, sp)

[Info] Get 60 instances


In [64]:
new_data = {
    'dict':{
        'src': word2idx,
    },
    'other_test':{
        'src': train_seq2seq_src,
        'tgt': train_seq2seq_tgt,
        'lex': train_seq2seq_lex
    }
}

In [65]:
torch.save(new_data,"/share/fangqiang/naacl/clean_data/zijizao_diff.pt")